In [41]:
# !rm -rf deconv_ztf/

In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2
scikit-learn-intelex==2024.1.0


See https://irsa.ipac.caltech.edu/docs/program_interface/ztf_metadata.html

- sciimg.fits
- mskimg.fits
- sciimgdaopsfcent.fits
- sciimgdao.psf (only for spatial variation of PSF)
- refimg.fits


```
R filter
l~0,b~0:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2021/0502/476053/ztf_20210502476053_000233_zr_c15_o_q3_sciimg.fits
l~100,b~10:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0706/370127/ztf_20220706370127_000829_zr_c10_o_q4_sciimg.fits (seeing ~ 1.84453 arcsec)
l~50,b~30:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2023/1021/119745/ztf_20231021119745_000635_zr_c12_o_q4_sciimg.fits
l~200,b~45:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2023/0127/346250/ztf_20230127346250_000619_zr_c11_o_q2_sciimg.fits
l~300,b~60:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2020/0306/404421/ztf_20200306404421_000423_zr_c07_o_q1_sciimg.fits
l~150,b~90:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/1120/493183/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits (seeing ~ 3.14735 arcsec)
Dwarf galaxy (NGC1569):https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2021/0220/218785/ztf_20210220218785_000811_zr_c16_o_q4_sciimg.fits

I filter
l~100,b~10:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0831/366910/ztf_20220831366910_000829_zi_c10_o_q4_sciimg.fits (seeing ~ 1.82194 arcsec)
l~150,b~90:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0301/372245/ztf_20220301372245_000626_zi_c12_o_q2_sciimg.fits (seeing ~ 3.31418 arcsec)
Globular cluster
(NGC 7006; Class 1 GC = highly concentrated; this is the only image which is not a 3kx3k field image, it is 300x300 arcsec. Use 9 subdivisions for this case so that the center is almost properly contained in the center subdivision): https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2019/0629/424757/ztf_20190629424757_000594_zr_c04_o_q1_sciimg.fits

G filter
l~100,b~10:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0901/326748/ztf_20220901326748_000829_zg_c10_o_q4_sciimg.fits (seeing ~ 1.91277 arcsec)
l~150,b~90:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/1126/507882/ztf_20221126507882_000626_zg_c12_o_q2_sciimg.fits (seeing ~ 3.10998 arcsec)


251 ID
l~200, b~ -60: https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2018/0831/500891/ztf_20180831500891_000251_zr_c16_o_q4_sciimg.fits
```

In [2]:
!python --version
!lscpu

Python 3.10.13
Architecture:                         x86_64
CPU op-mode(s):                       32-bit, 64-bit
Byte Order:                           Little Endian
Address sizes:                        46 bits physical, 48 bits virtual
CPU(s):                               4
On-line CPU(s) list:                  0-3
Thread(s) per core:                   2
Core(s) per socket:                   2
Socket(s):                            1
NUMA node(s):                         1
Vendor ID:                            GenuineIntel
CPU family:                           6
Model:                                79
Model name:                           Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                             0
CPU MHz:                              2200.210
BogoMIPS:                             4400.42
Hypervisor vendor:                    KVM
Virtualization type:                  full
L1d cache:                            64 KiB
L1i cache:                            64 KiB
L2 cache:   

In [42]:
!git clone https://<INSERT_TOKEN_HERE>@github.com/Yash-10/deconv_ztf.git

Cloning into 'deconv_ztf'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 265 (delta 18), reused 0 (delta 0), pack-reused 233 (from 1)
Receiving objects: 100% (265/265), 1.62 MiB | 9.32 MiB/s, done.
Resolving deltas: 100% (158/158), done.


In [43]:
!cp /kaggle/working/deconv_ztf/*.py /kaggle/working
!cp -r /kaggle/working/deconv_ztf/sgp_reconstruction_results /kaggle/working

# Imports

In [3]:
!pip install pyfftw
!pip install setuptools==69.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.0.3
    Uninstalling setuptools-69.0.3:
      Successfully uninstalled setuptools-69.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
conda 23.11.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.1 requires shapely>=2.0, but 

In [4]:
!pip install -U photutils
!pip install -U reproject

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 72.7 MB/s eta 0:00:00:00:0100:01
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5033 sha256=1196c80e03113f3776a6bab808b73dc6feb2e9bee265006fb9f94711a570ba4b
  Stored in directory: /root/.cache/pip/wheels/7f/4e/be/1171b40f43b918087657ec57cf3b81fa1a2e027d8755baa184
Successfully built asciitree


In [5]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
sns.set_style('white')
sns.set_context("paper", font_scale = 2)
sns.set(style='ticks')

import os
import numpy as np
import pandas as pd
import glob
from astropy.io import fits
from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import subprocess

from astropy.io import fits
from astropy.visualization import MinMaxInterval, ImageNormalize, SqrtStretch, ZScaleInterval, PercentileInterval, LogStretch

from scipy.stats import binned_statistic
from scipy.stats import median_abs_deviation

from IPython.display import Image

In [6]:
!sudo apt-get update
!sudo apt-get install -y source-extractor

Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:4 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]       
Get:5 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [27.8 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3285 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1273 kB]
Get:9 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1534 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]     
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4024 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4488 kB]
Get:13 htt

In [7]:
!source-extractor


         Source Extractor  version 2.25.0 (2018-02-08)

Written by Emmanuel BERTIN <bertin@iap.fr>
Copyright 2012-2015 IAP/CNRS/UPMC

visit http://astromatic.net/software/sextractor

Source Extractor comes with ABSOLUTELY NO WARRANTY
You may redistribute copies of Source Extractor
under the terms of the GNU General Public License.

> SYNTAX: source-extractor <image> [<image2>][-c <configuration_file>][-<keyword> <value>]
> to dump a default configuration file:          source-extractor -d 
> to dump a default extended configuration file: source-extractor -dd 
> to dump a full list of measurement parameters: source-extractor -dp 




In [8]:
# We use Topcat 4.9-1, which is the latest version as of writing (early April 2024).
# Earlier versions had some bugs in the symmetric crossmatching. So need to beware.
!wget --directory-prefix sgp_reconstruction_results https://www.star.bristol.ac.uk/mbt/releases/topcat/v4.9-1/topcat-full.jar
!wget --directory-prefix sgp_reconstruction_results https://www.star.bris.ac.uk/~mbt/topcat/topcat
%cd sgp_reconstruction_results
!chmod +x topcat
!./topcat -stilts -version
!./topcat -version
%cd ..

--2024-09-27 09:59:42--  https://www.star.bristol.ac.uk/mbt/releases/topcat/v4.9-1/topcat-full.jar
Resolving www.star.bristol.ac.uk (www.star.bristol.ac.uk)... 137.222.0.168
Connecting to www.star.bristol.ac.uk (www.star.bristol.ac.uk)|137.222.0.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37895037 (36M) [application/x-java-archive]
Saving to: 'sgp_reconstruction_results/topcat-full.jar'

topcat-full.jar     100%[===================>]  36.14M  1.81MB/s    in 24s     

2024-09-27 10:00:07 (1.53 MB/s) - 'sgp_reconstruction_results/topcat-full.jar' saved [37895037/37895037]

--2024-09-27 10:00:08--  https://www.star.bris.ac.uk/~mbt/topcat/topcat
Resolving www.star.bris.ac.uk (www.star.bris.ac.uk)... 137.222.0.168
Connecting to www.star.bris.ac.uk (www.star.bris.ac.uk)|137.222.0.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3850 (3.8K) [text/plain]
Saving to: 'sgp_reconstruction_results/topcat'

topcat              100%[=====

# Deconvolution run

For this notebook, uncomment one of the lines in the prefixes below on which deconvolution must be run.

In [9]:
prefixes = [
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2021/0502/476053/ztf_20210502476053_000233_zr_c15_o_q3_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0706/370127/ztf_20220706370127_000829_zr_c10_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2023/1021/119745/ztf_20231021119745_000635_zr_c12_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2023/0127/346250/ztf_20230127346250_000619_zr_c11_o_q2_',
#     'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2020/0306/404421/ztf_20200306404421_000423_zr_c07_o_q1_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/1120/493183/ztf_20221120493183_000626_zr_c12_o_q2_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2021/0220/218785/ztf_20210220218785_000811_zr_c16_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0831/366910/ztf_20220831366910_000829_zi_c10_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0301/372245/ztf_20220301372245_000626_zi_c12_o_q2_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0901/326748/ztf_20220901326748_000829_zg_c10_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/1126/507882/ztf_20221126507882_000626_zg_c12_o_q2_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2018/0831/500891/ztf_20180831500891_000251_zr_c16_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2019/0629/424757/ztf_20190629424757_000594_zr_c04_o_q1_'
]

image_names = [
    'sciimg.fits',
    'mskimg.fits',
    'sciimgdaopsfcent.fits',
    'sciimgdao.psf',
    # 'refimg.fits'
]

In [1]:
%%capture
for prefix in prefixes:
    for image_name in image_names:
        name = prefix + image_name
        result = subprocess.run(['wget', f'{name}'], stdout=subprocess.DEVNULL);
        print(result)

The folder `sgp_reconstruction_results` requires the following files to be present before running `run.py`:

- `default.conv`
- `default.param`
- `default.nnw`
- File provided as argument to `sextractor_config_file_name` below if using SExtractor instead of sep. (NOTE: The sep version is not tested as effectively as the SExtractor version).

In [11]:
# !python3 run.py --data_path_sciimg /kaggle/working/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits --data_path_psf /kaggle/working/ztf_20221120493183_000626_zr_c12_o_q2_sciimgdaopsfcent.fits --init_recon 2 --stop_criterion 3 --use_subdiv --subdiv_size 512 --subdiv_overlap 10 --use_sextractor --sextractor_config_file_name sextractor_config.fits_scat_sextractor.sex --use_subdiv --interpolate_bad_pixels --tol_convergence 1e-4 --reconstruct_subdivisions_fast --perform_catalog_crossmatching

In [2]:
from subprocess import Popen, PIPE, CalledProcessError, STDOUT
def execute(cmd):
    """This function is taken from
    https://stackoverflow.com/a/28319191
    """
#     logfile = open('sgp_run_all.log', 'a')
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True, stderr=STDOUT) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
#         print(p.output)
        raise CalledProcessError(p.returncode, p.args)

for prefix in prefixes:
    if 'ztf_20190629424757_000594_zr_c04_o_q1_' in prefix:  # Means it's a globular cluster. We are using the same command as the non-globular cluster case (previously I used subdiv_size=128).
        command = [
            'python3', 'run.py', '--data_path_sciimg', f'/kaggle/working/{prefix.split("/")[-1]}sciimg.fits',
            '--data_path_psf', f'/kaggle/working/{prefix.split("/")[-1]}sciimgdaopsfcent.fits',
            '--init_recon', '2', '--stop_criterion', '3', '--use_subdiv', '--subdiv_size', '512', '--subdiv_overlap', '10',
            '--use_sextractor', '--sextractor_config_file_name', 'sextractor_config.fits_scat_sextractor.sex',
            '--use_subdiv', '--interpolate_bad_pixels', '--reconstruct_full_image_from_subdivisions',
            '--tol_convergence', '1e-4',
            # '--pixel_mask', '/kaggle/working/ztf_20180831500891_000251_zr_c16_o_q4_mskimg.fits',  # todo: how to use this?
            '--perform_catalog_crossmatching'
        ]
    else:
        command = [
            'python3', 'run.py', '--data_path_sciimg', f'/kaggle/working/{prefix.split("/")[-1]}sciimg.fits',
            '--data_path_psf', f'/kaggle/working/{prefix.split("/")[-1]}sciimgdaopsfcent.fits',
            '--init_recon', '2', '--stop_criterion', '3', '--use_subdiv', '--subdiv_size', '512', '--subdiv_overlap', '10',
            '--use_sextractor', '--sextractor_config_file_name', 'sextractor_config.fits_scat_sextractor.sex',
            '--use_subdiv', '--interpolate_bad_pixels', '--reconstruct_full_image_from_subdivisions',
            '--tol_convergence', '1e-4',
            # '--pixel_mask', '/kaggle/working/ztf_20180831500891_000251_zr_c16_o_q4_mskimg.fits',  # todo: how to use this?
            '--perform_catalog_crossmatching'
        ]

    execute(command)

In [ ]:
# from astropy.io import fits
# from astropy.visualization import MinMaxInterval, ImageNormalize, SqrtStretch, ZScaleInterval, PercentileInterval, LogStretch

# arr = fits.getdata('sgp_reconstruction_results/deconvolved_subdiv_rearranged_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits')
# norm = ImageNormalize(arr, interval=PercentileInterval(99.),
#                       stretch=SqrtStretch())
# plt.imshow(arr, norm=norm)

In [ ]:
# # df.groupby(by='ID_PARENT')
# for name, group in df.groupby(by='ID_PARENT'):
# #     if len(group) > 1:
# #         print(group[['ID_PARENT', 'NUMBER']])
#     if len(group) == 1:
#         print(group[['ID_PARENT', 'NUMBER']])
# #         break

## Outro

Need to upload kaggle.json file manually.

In [ ]:
# DIRNAME = 'sgp_reconstruction_results'

# def cut_condition(cat):
#     return (cat['MAG_ISO'] >= 15) & (cat['MAG_ISO'] <= 20.5) & (cat['FWHM_IMAGE'] >= 0) & (cat['FWHM_IMAGE'] <= 4) & (cat['ELLIPTICITY'] < 0.5)

# def cut_condition2(cat):
#     return (cat['MAG_ISO_2'] >= 15) & (cat['MAG_ISO_2'] <= 20.5) & (cat['FWHM_IMAGE_2'] >= 0) & (cat['FWHM_IMAGE_2'] <= 4) & (cat['ELLIPTICITY_2'] < 0.5)

# def cut_condition2_one_to_one(cat):
#     assert cat['MAG_ISO_2_x'].equals(cat['MAG_ISO_2_y'])
#     assert cat['FWHM_IMAGE_2_x'].equals(cat['FWHM_IMAGE_2_y'])
#     assert cat['ELLIPTICITY_2_x'].equals(cat['ELLIPTICITY_2_y'])
#     return (cat['MAG_ISO_2_x'] >= 15) & (cat['MAG_ISO_2_x'] <= 20.5) & (cat['FWHM_IMAGE_2_x'] >= 0) & (cat['FWHM_IMAGE_2_x'] <= 4) & (cat['ELLIPTICITY_2_x'] < 0.5)

# color1 = '#FC9272'
# color2 = '#3182bd'
# # ip_gen_color = '#2ca25f'

# ticklabelsize = 21
# axeslabelsize = 22
# titlesize = 25

# linewidth = 4
# alpha = 0.4
# scatter_point_size = 30

# from sklearn.metrics import r2_score

In [ ]:
# for counter, prefix in enumerate(prefixes):
#     if '_zr_' not in prefix:
#         continue
#     title = prefix.split('/')[-1].split('_')[2][3:]
#     CROSSMATCH_PREFIX = f'{prefix.split("/")[-1]}sciimg.fits'
#     ID = prefix.split('/')[-1].split('_')[2][3:]

#     IMG_NAMES = [
#         f'sgp_reconstruction_results/orig_subdiv_{CROSSMATCH_PREFIX}',
#     ]

#     # 2.
#     SOURCE_CATALOGS = [
#         f'orig_{CROSSMATCH_PREFIX}_scat_sextractor.csv', f'deconv_{CROSSMATCH_PREFIX}_scat_sextractor.csv'
#     ]

#     # Remember: In crossmatched catalogs, 1 is deconvolved and 2 means original.
#     # Topcat settings: 2-d cartesian, 1.383 error, and X_IMAGE_DBL and Y_IMAGE_DBL are used.
#     CROSSMATCHED_CATALOG_best_match_for_each_table2_row_1_and_2 = f'{CROSSMATCH_PREFIX}_crossmatched_best_match_for_each_table2_row_1_and_2_stilts.csv'
#     CROSSMATCHED_CATALOG_best_match_for_each_table1_row_1_and_2 = f'{CROSSMATCH_PREFIX}_crossmatched_best_match_for_each_table1_row_1_and_2_stilts.csv'
#     CROSSMATCHED_CATALOG_best_match_symmetric_1_and_2 = f'{CROSSMATCH_PREFIX}_crossmatched_best_match_symmetric_1_and_2_stilts.csv'
#     CROSSMATCHED_CATALOG_all_matches = f'{CROSSMATCH_PREFIX}_crossmatched_all_matches_1_and_2_stilts.csv'

#     CROSSMATCHED_CATALOGS = [
#         CROSSMATCHED_CATALOG_best_match_for_each_table2_row_1_and_2,
#         CROSSMATCHED_CATALOG_best_match_symmetric_1_and_2,
#         CROSSMATCHED_CATALOG_best_match_for_each_table1_row_1_and_2,
#         CROSSMATCHED_CATALOG_all_matches
#     ]
#     CROSSMATCHED_CATALOGS_NAMES = [
#         'best_match_for_each_table2_row_1_and_2',
#         'best_match_symmetric_1_and_2',
#         'best_match_for_each_table1_row_1_and_2',
#         'all_matches_1_and_2'
#     ]

#     CROSSMATCHED_CATALOGS_2_not_1 = f'{CROSSMATCH_PREFIX}_crossmatched_best_match_for_each_table2_row_2_not_1_stilts.csv'
#     CROSSMATCHED_CATALOGS_1_not_2 = f'{CROSSMATCH_PREFIX}_crossmatched_best_match_for_each_table1_row_1_not_2_stilts.csv'

#     IMAGE_FITS_NAMES = [
#         f'orig_subdiv_{CROSSMATCH_PREFIX}', f'deconvolved_subdiv_{CROSSMATCH_PREFIX}'
#     ]

#     NAMES = [CROSSMATCH_PREFIX] * len(CROSSMATCHED_CATALOGS)

#     catname = CROSSMATCHED_CATALOGS
#     mcat1 = pd.read_csv(os.path.join(DIRNAME, catname[0]))  # best_match_for_each_table2_row_1_and_2 (Each row from table 2 will appear a maximum of once in the result, but rows from table 1 may appear multiple times.)
#     mcat2 = pd.read_csv(os.path.join(DIRNAME, catname[1]))  # best_match_symmetric_1_and_2
#     mcat3 = pd.read_csv(os.path.join(DIRNAME, catname[2]))  # best_match_for_each_table1_row_1_and_2 (Each row from table 1 will appear a maximum of once in the result, but rows from table 2 may appear multiple times.)
#     # mcat4 is all matches, but that is not used for the plot here.
#     mcat4 = pd.read_csv(os.path.join(DIRNAME, catname[3]))  # all_matches

#     catname = [CROSSMATCHED_CATALOGS_2_not_1, CROSSMATCHED_CATALOGS_1_not_2, CROSSMATCHED_CATALOGS[2], SOURCE_CATALOGS]
#     um_ocat = pd.read_csv(os.path.join(DIRNAME, catname[0]))
#     um_dcat = pd.read_csv(os.path.join(DIRNAME, catname[1]))
#     ocat = pd.read_csv(os.path.join(DIRNAME, catname[3][0]))
#     dcat = pd.read_csv(os.path.join(DIRNAME, catname[3][1]))

#     many_to_one = []
#     one_to_one1 = []
#     for mcat3_col1_2 in mcat3['col1_2'].unique():
#         mcat3_subset = mcat3[mcat3['col1_2'] == mcat3_col1_2]
#         if len(mcat3_subset) == 1:
#             one_to_one1.append(mcat3_subset)
#         elif len(mcat3_subset) > 1:
#             many_to_one.append(mcat3_subset)
#             # one_to_one1.append(mcat3_subset[mcat3_subset['Separation'] == mcat3_subset['Separation'].min()])

#     one_to_many = []
#     one_to_one2 = []
#     for mcat1_col1_1 in mcat1['col1_1'].unique():
#         mcat1_subset = mcat1[mcat1['col1_1'] == mcat1_col1_1]
#         if len(mcat1_subset) == 1:
#             one_to_one2.append(mcat1_subset)
#         elif len(mcat1_subset) > 1:
#             one_to_many.append(mcat1_subset)
#             # one_to_one2.append(mcat1_subset[mcat1_subset['Separation'] == mcat1_subset['Separation'].min()])

#     # one_to_one = pd.merge(pd.concat((one_to_one1)), pd.concat((one_to_one2)), how='inner', on=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2', 'X_IMAGE_DBL_1', 'Y_IMAGE_DBL_1'])

#     one_to_one1 = pd.concat((one_to_one1))
#     one_to_one2 = pd.concat((one_to_one2))
#     # NOTE: One can use the merge option to create one to one as well, but perhaps the concat followed by removal of duplicates is better suited.
#     # one_to_one = pd.merge(one_to_one1, one_to_one2, how='inner', on=['col1_2', 'col1_1'])
#     one_to_one = pd.concat((one_to_one1, one_to_one2)).drop_duplicates(subset=['col1_2', 'col1_1'])

#     # assert len(one_to_one) == len(mcat2)  # NOTE: This wouldn't be equal to mcat2 unless we uncomment the below two lines above:
#     """
#     # one_to_one1.append(mcat3_subset[mcat3_subset['Separation'] == mcat3_subset['Separation'].min()])
#     # one_to_one2.append(mcat1_subset[mcat1_subset['Separation'] == mcat1_subset['Separation'].min()])
#     """

#     many_to_one = pd.DataFrame(np.vstack((many_to_one)))
#     many_to_one.columns = mcat3.columns
#     one_to_many = pd.DataFrame(np.vstack((one_to_many)))
#     one_to_many.columns = mcat1.columns

#     # many_to_one = many_to_one.drop_duplicates(subset=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2'])
#     # one_to_many = one_to_many.drop_duplicates(subset=['X_IMAGE_DBL_1', 'Y_IMAGE_DBL_1'])

#     ocat_cut = ocat[cut_condition(ocat)]
#     one_to_one_cut = one_to_one[cut_condition2(one_to_one)]
#     many_to_one_cut = many_to_one[cut_condition2(many_to_one)]
#     one_to_many_cut = one_to_many[cut_condition2(one_to_many)]
#     unmatches_original_cut = um_ocat[cut_condition(um_ocat)]

#     # x = pd.merge(
#     #     many_to_one.drop_duplicates(subset=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2']),
#     #     one_to_one.drop_duplicates(subset=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2']),
#     #     on=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2']

#     # )
#     # y = pd.merge(
#     #     many_to_one.drop_duplicates(subset=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2']),
#     #     one_to_many.drop_duplicates(subset=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2']),
#     #     on=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2']

#     # )
#     # z = pd.merge(
#     #     one_to_many.drop_duplicates(subset=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2']),
#     #     one_to_one.drop_duplicates(subset=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2']),
#     #     on=['X_IMAGE_DBL_2', 'Y_IMAGE_DBL_2']

#     # )

#     # assert one_to_one['col1_2_x'].equals(one_to_one['col1_2_y'])

#     x2 = pd.merge(
#         many_to_one.drop_duplicates(subset=['col1_2']),
#         one_to_one.drop_duplicates(subset=['col1_2']),#.rename(columns={'col1_2_x': 'col1_2'}),
#         on=['col1_2']
#     )
#     y2 = pd.merge(
#         many_to_one.drop_duplicates(subset=['col1_2']),
#         one_to_many.drop_duplicates(subset=['col1_2']),
#         on=['col1_2']
#     )
#     z2 = pd.merge(
#         one_to_many.drop_duplicates(subset=['col1_2']),
#         one_to_one.drop_duplicates(subset=['col1_2']),#.rename(columns={'col1_2_x': 'col1_2'}),
#         on=['col1_2']
#     )
#     x1 = pd.merge(
#         many_to_one.drop_duplicates(subset=['col1_1']),
#         one_to_one.drop_duplicates(subset=['col1_1']),#.rename(columns={'col1_2_x': 'col1_2'}),
#         on=['col1_1']
#     )
#     y1 = pd.merge(
#         many_to_one.drop_duplicates(subset=['col1_1']),
#         one_to_many.drop_duplicates(subset=['col1_1']),
#         on=['col1_1']
#     )
#     z1 = pd.merge(
#         one_to_many.drop_duplicates(subset=['col1_1']),
#         one_to_one.drop_duplicates(subset=['col1_1']),#.rename(columns={'col1_2_x': 'col1_2'}),
#         on=['col1_1']
#     )
#     # xyz = pd.merge(
#     #     one_to_many.drop_duplicates(subset=['col1_2']),
#     #     one_to_one.drop_duplicates(subset=['col1_2']),#.rename(columns={'col1_2_x': 'col1_2'}),
#     #     on=['col1_2']
#     # ).merge(
#     #     one_to_one.drop_duplicates(subset=['col1_2']),
#     #     on=['col1_2']
#     # )

#     w1 = pd.merge(
#         one_to_many.drop_duplicates(subset=['col1_2']),
#         um_ocat.drop_duplicates(subset=['col1']).rename(columns={'X_IMAGE_DBL': 'X_IMAGE_DBL_2', 'Y_IMAGE_DBL': 'Y_IMAGE_DBL_2', 'col1': 'col1_2'}),
#         on=['col1_2']
#     )
#     w2 = pd.merge(
#         one_to_one.drop_duplicates(subset=['col1_2']),
#         um_ocat.drop_duplicates(subset=['col1']).rename(columns={'X_IMAGE_DBL': 'X_IMAGE_DBL_2', 'Y_IMAGE_DBL': 'Y_IMAGE_DBL_2', 'col1': 'col1_2'}),
#         on=['col1_2']
#     )
#     w3 = pd.merge(
#         many_to_one.drop_duplicates(subset=['col1_2']),
#         um_ocat.drop_duplicates(subset=['col1']).rename(columns={'X_IMAGE_DBL': 'X_IMAGE_DBL_2', 'Y_IMAGE_DBL': 'Y_IMAGE_DBL_2', 'col1': 'col1_2'}),
#         on=['col1_2']
#     )
#     assert len(w1) == 0
#     assert len(w2) == 0
#     assert len(w3) == 0

#     if not len(ocat) == len(one_to_one) + len(um_ocat) + len(many_to_one['col1_2'].unique()) + len(one_to_many['col1_2'].unique()) - (len(x2) + len(y2) + len(z2)):
#         print(f"ocat assertion error for {title}\t{len(ocat)} vs. {len(one_to_one) + len(um_ocat) + len(many_to_one['col1_2'].unique()) + len(one_to_many['col1_2'].unique()) - (len(x2) + len(y2) + len(z2))}")
#     if not len(dcat) == len(one_to_one) + len(um_dcat) + len(many_to_one['col1_1'].unique()) + len(one_to_many['col1_1'].unique()) - (len(x1) + len(y1) + len(z1)):
#         print(f"dcat assertion error for {title}\t{len(dcat)} vs. {len(one_to_one) + len(um_dcat) + len(many_to_one['col1_1'].unique()) + len(one_to_many['col1_1'].unique()) - (len(x1) + len(y1) + len(z1))}")
#     # print(len(xyz), len(x), len(y), len(z), len(many_to_one['col1_2'].unique()), len(one_to_many['col1_2'].unique()), len(um_ocat['col1'].unique()), len(w1), len(w2), len(w3))

#     ############### Assertion for one to many and many to one matches ###############
#     for m2o_col1_2 in many_to_one['col1_2'].unique():
#         many_to_one_subset = many_to_one[many_to_one['col1_2'] == m2o_col1_2]
#         assert len(many_to_one_subset) > 1

#     for o2m_col1_1 in one_to_many['col1_1'].unique():
#         one_to_many_subset = one_to_many[one_to_many['col1_1'] == o2m_col1_1]
#         assert len(one_to_many_subset) > 1
#     #################################################################################

#     fig, ax = plt.subplots(2, 1, figsize=(7.5, 7), sharex=True, gridspec_kw={'height_ratios': [2, 1]})
#     fig.subplots_adjust(hspace=0)

#     mo = one_to_one_cut['MAG_ISO_2']
#     md = one_to_one_cut['MAG_ISO_1']

#     r2 = r2_score(mo, md)

#     ax[0].scatter(mo, md, alpha=alpha, s=scatter_point_size, c=color2, label=fr'$R^2$ = {np.round(r2, 2)}')
#     ax[0].plot(mo, mo, c='black')
#     # hb = ax[0].hexbin(mo, md, bins='log', gridsize=50, cmap='inferno')
#     # ax[0].legend(fontsize=ticklabelsize)
#     # ax[0].set_title('Mean Power spectra', fontsize=titlesize)
#     ax[0].tick_params(axis='x', labelsize=ticklabelsize)
#     ax[0].tick_params(axis='y', labelsize=ticklabelsize)
#     ax[0].set_title(title+' (1-1)', fontsize=titlesize)
#     l = ax[0].legend(fontsize=axeslabelsize, frameon=False, loc='upper left', handletextpad=-1.5)

#     for item in l.legend_handles:
#         item.set_visible(False)

#     # if counter == 0:
#     #     ax[0].set_ylabel(r'Deconvolved mag ($m_{\mathrm{d}}$)', fontsize=axeslabelsize)
#     #     ax[1].set_xlabel(r'Original mag ($m_{\mathrm{o}}$)', fontsize=axeslabelsize)
#     # else:
#     ax[0].set_ylabel(r'$m_{\mathrm{d}}$', fontsize=axeslabelsize)
#     ax[1].set_xlabel(r'$m_{\mathrm{o}}$', fontsize=axeslabelsize)

#     # ax[0].set_yticks([100, 1000])
#     # ax[0].set_ylim([10, 1000])

#     ax[1].scatter(mo, md - mo, c=color2, s=scatter_point_size, alpha=alpha, linewidth=linewidth)
#     ax[1].axhline(y=0, c='black', linestyle='--', linewidth=linewidth-2, alpha=0.9)
#     # ax[1].axhline(y=5, c='black', linestyle='dashed', linewidth=linewidth-2, alpha=0.9)
#     # ax[1].set_ylabel('$m_{\mathrm{d}} - m_{\mathrm{o}}$', fontsize=axeslabelsize)
#     ax[1].set_ylabel('$\Delta m$', fontsize=axeslabelsize)
#     ax[1].tick_params(axis='x', labelsize=ticklabelsize)
#     ax[1].tick_params(axis='y', labelsize=ticklabelsize)
#     ax[1].fill_between([mo.min(), mo.max()], -0.5, 0.5, alpha=0.3, color='gray')  # corresponds to ~90% flux conservation.
#     # ax[0].set_xlim([k.min(), 5])
#     # ax[1].set_xlim([k.min(), 5])
#     ax[1].set_ylim([-1.5, +1.5])
#     ax[1].set_yticks(np.arange(-1, +1.5, 0.5))

#     for a in ax:
#         a.set_xlim([mo.min(), mo.max()])

# #     plt.savefig(f'mag_one_to_one_comparison_{title}.pdf', bbox_inches='tight', dpi=200, format='pdf')
#     plt.show()

#     ######################### FIGURE 5: MAG_ISO (MANY-1) comparison #########################
#     def mag_to_flux(m, magzp):
#         # magzp = image_header['MAGZP']
#         return 10 ** (0.4 * (magzp - m))
#     def flux_to_mag(flux, magzp):
#         m = magzp - 2.5 * np.log10(flux) if flux > 0 else 99
#         return m

#     fig, ax = plt.subplots(2, 1, figsize=(7.5, 7), sharex=True, gridspec_kw={'height_ratios': [2, 1]})
#     fig.subplots_adjust(hspace=0)

#     fluxoM2O, fluxdM2O = [], []
#     moM2O, mdM2O = [], []
#     hdul = fits.open(CROSSMATCH_PREFIX)
#     image_header = hdul[0].header
#     magzp = image_header['MAGZP']
#     for m2o_col1_2 in many_to_one['col1_2'].unique():
#         many_to_one_subset = many_to_one[many_to_one['col1_2'] == m2o_col1_2]
# #         many_to_one_subset = many_to_one_subset[many_to_one_subset['Separation'] < 0.5]
#         # TODO: Should we select more stringent subset? using separation < 0.5 or something?? then we kind of ensure these are true deblended cases. We can also instead of 0.5 use the value based on 0.2 quantile of all separations from many to one dataframe, for eg.
#         assert len(many_to_one_subset['col1_1'].unique()) == len(many_to_one_subset['col1_1'])
#         # assert len(many_to_one_subset['col1_1'].unique()) > 1
# #         if len(many_to_one_subset) == 0:
# #             continue
# #         if len(many_to_one_subset) == 1:  # Only if Separation condition is True.
# #             fluxoM2O_current = mag_to_flux(many_to_one_subset['MAG_ISO_2'].iloc[0], magzp)
# #             fluxdM2O_current = many_to_one_subset['MAG_ISO_1'].apply(mag_to_flux, magzp=magzp).sum()
# #             moM2O_current = flux_to_mag(fluxoM2O_current, magzp)
# #             mdM2O_current = flux_to_mag(fluxdM2O_current, magzp)
# #             # print(moM2O_current, mdM2O_current)
# #             assert np.allclose(moM2O_current, many_to_one_subset['MAG_ISO_2'].iloc[0])
# #         else:
#         fluxoM2O_current = mag_to_flux(many_to_one_subset['MAG_ISO_2'].unique()[0], magzp)
#         fluxdM2O_current = mag_to_flux(
#             many_to_one_subset[many_to_one_subset['Separation'] == many_to_one_subset['Separation'].min()]['MAG_ISO_1'].iloc[0],
#             magzp
#         )
#         moM2O_current = flux_to_mag(fluxoM2O_current, magzp)
#         mdM2O_current = flux_to_mag(fluxdM2O_current, magzp)
#         # print(moM2O_current, mdM2O_current)
# #         print(many_to_one_subset['MAG_ISO_1'])
#         assert np.allclose(moM2O_current, many_to_one_subset['MAG_ISO_2'].unique()[0])
#         moM2O.append(moM2O_current)
#         mdM2O.append(mdM2O_current)

#     moM2O = np.array(moM2O)
#     mdM2O = np.array(mdM2O)

#     r2 = r2_score(moM2O, mdM2O)

#     ax[0].scatter(moM2O, mdM2O, alpha=alpha, s=scatter_point_size, c=color2, label=fr'$R^2$ = {np.round(r2, 2)}')
#     ax[0].plot(moM2O, moM2O, c='black')
#     # hb = ax[0].hexbin(mo, md, bins='log', gridsize=50, cmap='inferno')
#     # ax[0].legend(fontsize=ticklabelsize)
#     # ax[0].set_title('Mean Power spectra', fontsize=titlesize)
#     ax[0].tick_params(axis='x', labelsize=ticklabelsize)
#     ax[0].tick_params(axis='y', labelsize=ticklabelsize)
#     ax[0].set_title(title + ' (M-1)', fontsize=titlesize)
#     l = ax[0].legend(fontsize=axeslabelsize, frameon=False, loc='upper left', handletextpad=-1.5)

#     for item in l.legend_handles:
#         item.set_visible(False)

#     # if counter == 0:
#     #     ax[0].set_ylabel(r'Deconvolved mag ($m_{\mathrm{d}}$)', fontsize=axeslabelsize)
#     #     ax[1].set_xlabel(r'Original mag ($m_{\mathrm{o}}$)', fontsize=axeslabelsize)
#     # else:
#     ax[0].set_ylabel(r'$m_{\mathrm{d}}$', fontsize=axeslabelsize)
#     ax[1].set_xlabel(r'$m_{\mathrm{o}}$', fontsize=axeslabelsize)

#     # ax[0].set_yticks([100, 1000])
#     # ax[0].set_ylim([10, 1000])

#     ax[1].scatter(moM2O, mdM2O - moM2O, c=color2, s=scatter_point_size, alpha=alpha, linewidth=linewidth)
#     ax[1].axhline(y=0, c='black', linestyle='--', linewidth=linewidth-2, alpha=0.9)
#     # ax[1].axhline(y=5, c='black', linestyle='dashed', linewidth=linewidth-2, alpha=0.9)
#     # ax[1].set_ylabel('$m_{\mathrm{d}} - m_{\mathrm{o}}$', fontsize=axeslabelsize)
#     ax[1].set_ylabel('$\Delta m$', fontsize=axeslabelsize)
#     ax[1].tick_params(axis='x', labelsize=ticklabelsize)
#     ax[1].tick_params(axis='y', labelsize=ticklabelsize)
#     ax[1].fill_between([moM2O.min(), moM2O.max()], -0.5, 0.5, alpha=0.3, color='gray')  # corresponds to ~90% flux conservation.
#     # ax[0].set_xlim([k.min(), 5])
#     # ax[1].set_xlim([k.min(), 5])
#     ax[1].set_ylim([-1.5, +1.5])
#     ax[1].set_yticks(np.arange(-1, +1.5, 0.5))

#     for a in ax:
#         a.set_xlim([moM2O.min(), moM2O.max()])

#     plt.show()
# #     plt.savefig(f'mag_many_to_one_comparison_{title}.pdf', bbox_inches='tight', dpi=200, format='pdf')
#     # plt.clf()
#     ###################################################################################

## reference images

In [46]:
%%capture

import subprocess
# reference: see 8b in https://irsa.ipac.caltech.edu/data/ZTF/docs/releases/ztf_release_notes_latest
ref_names = [
#     'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/ref/000/field000626/zr/ccd12/q2/ztf_000626_zr_c12_q2_refimg.fits',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/deep/000/field000626/zr/ccd12/q2/ztf_000626_zr_c12_q2_refimg.fits'
]
for name in ref_names:
    result = subprocess.run(['wget', f'{name}'])#, stdout=subprocess.DEVNULL);
    print(result)

--2024-09-24 12:01:25--  https://irsa.ipac.caltech.edu/ibe/data/ztf/products/deep/000/field000626/zr/ccd12/q2/ztf_000626_zr_c12_q2_refimg.fits
Resolving irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)... 134.4.54.87
Connecting to irsa.ipac.caltech.edu (irsa.ipac.caltech.edu)|134.4.54.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40970880 (39M) [application/octet-stream]
Saving to: 'ztf_000626_zr_c12_q2_refimg.fits'

     0K .......... .......... .......... .......... ..........  0%  166K 4m0s
    50K .......... .......... .......... .......... ..........  0%  332K 3m0s
   100K .......... .......... .......... .......... ..........  0% 76.3M 2m0s
   150K .......... .......... .......... .......... ..........  0%  333K 2m0s
   200K .......... .......... .......... .......... ..........  0% 53.5M 96s
   250K .......... .......... .......... .......... ..........  0%  334K 1m40s
   300K .......... .......... .......... .......... ..........  0% 98.0M 85s
   350

In [6]:

import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord
import astropy.units as u

fits.open('/kaggle/working/ztf_000626_zr_c12_q2_refimg.fits')[0].header['MAGLIM']

22.95

In [9]:

import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord
import astropy.units as u

fits.open('/kaggle/working/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits')[0].header['MAGLIM']

20.27

In [3]:
# !python ref_sci_crossmatch.py

import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord
import astropy.units as u

from utils import create_subdivisions, source_info, remove_very_close_coords
from constants import CAT_COLUMNS

# CAT_COLUMNS2 = CAT_COLUMNS.copy()

data_path_refimg = '/kaggle/input/ztf-deconvolution-ref-and-sci/ztf_000626_zr_c12_q2_deeprefimg_align1.fits'
data_path_sciimg = '/kaggle/input/ztf-deconvolution-ref-and-sci/ztf_20221120493183_000626_zr_c12_o_q2_sciimg_align1.fits'
ref = fits.open(data_path_refimg)[0]
sci = fits.open(data_path_sciimg)[0]
ref_header = ref.header
sci_header = sci.header

basename = data_path_refimg.split('/')[-1]

# Options
subdiv_size = 512
use_sextractor = True
use_subdiv = True
subdiv_overlap = 10
sextractor_config_file_name = 'sextractor_config.fits_scat_sextractor.sex'
sextractor_parameters = {
    'MAG_ZEROPOINT': ref_header['MAGZP'],
    'GAIN': ref_header['GAIN'] if 'GAIN' in ref_header.keys() else sci_header['GAIN'],
    'SEEING_FWHM': ref_header['SEEING'] * ref_header['PIXSCALE'] if 'SEEING' in ref_header.keys() and 'PIXSCALE' in ref_header.keys() else 2.5,  # 2.5 is from ztf config file.
    'PIXEL_SCALE': ref_header['PIXSCALE'] if 'PIXSCALE' in ref_header.keys() else sci_header['PIXSCALE']
}

wcs = WCS(ref.header)
subdivs = create_subdivisions(
    ref.data, subdiv_shape=(subdiv_size, subdiv_size),
    overlap=subdiv_overlap, wcs=wcs
)

dirname = 'sgp_reconstruction_results'
dtype = np.float32
orig_objects = []

for i, subdiv in enumerate(subdivs):
    assert subdiv.data.shape == (subdiv_size, subdiv_size)
    if use_sextractor:
        fits.writeto(f'{dirname}/subdiv_{i}_temp.fits', subdiv.data.astype(dtype), overwrite=True)

    # This is added because unlike the run from github code, cat_columns scope is also used within source_info...
    from utils import source_info
#     from constants import CAT_COLUMNS
    os.chdir('/kaggle/working')
    objects, orig_fluxes_subdiv, orig_bkg, orig_bkg_rms, fig = source_info(
        ref.data, subdiv_size, subdiv_size,
        use_sextractor=use_sextractor, image_name=f'subdiv_{i}_temp.fits',
        defaultFile=sextractor_config_file_name, i=i,
        sextractor_parameters=sextractor_parameters, original=True, use_subdiv=True
    )

    if objects is None:
        print(f'\n\nNo source detected in subdivision {i}\n\n')
        # Note that even if no source is detected, we need to still run deconvolution.
        # Since maybe the source existed but was just not detected by SExtractor.
        # The deconvolution may then find one or more new sources if indeed there was a source.
        objects = pd.DataFrame(np.expand_dims(np.ones(len(CAT_COLUMNS)), 0) * -99)
        objects.columns = CAT_COLUMNS
        objects[np.where(np.array(CAT_COLUMNS) == 'FLUX_ISO')[0][0]] = 0.0
        objects[np.where(np.array(CAT_COLUMNS) == 'FLUXERR_ISO')[0][0]] = 0.0
        objects[np.where(np.array(CAT_COLUMNS) == 'BACKGROUND')[0][0]] = 0.0

        # Since there is no source, we cannot sum the fluxes from the SExtractor catalog.
        # Hence, we give a simple flux estimate by sum(pixels - bkg).
        orig_fluxes_subdiv = np.sum(ref.data - orig_bkg)

    x_in_nonsubdivided = []
    y_in_nonsubdivided = []
    xpeak_in_nonsubdivided = []
    ypeak_in_nonsubdivided = []
    ra_in_nonsubdivided = []
    dec_in_nonsubdivided = []

    for obj in objects.itertuples():
        # NOTE: 1 is subtracted from the detected coordinates since photutils follows the (0, 0) convention
        # whereas SExtractor follows the (1, 1) convention. See https://github.com/astropy/photutils/blob/main/docs/pixel_conventions.rst
        _x, _y = subdiv.to_original_position((obj.X_IMAGE_DBL-1, obj.Y_IMAGE_DBL-1))
        _xpeak, _ypeak = subdiv.to_original_position((obj.XPEAK_IMAGE-1, obj.YPEAK_IMAGE-1))
        # No need of origin=1 to get 1-based pixel coordinates since 1 is subtracted above already.
        celestial_coords = pixel_to_skycoord(_x, _y, wcs)
        celestial_coords_peak = pixel_to_skycoord(_xpeak, _ypeak, wcs)
        x_in_nonsubdivided.append(_x)
        y_in_nonsubdivided.append(_y)
        xpeak_in_nonsubdivided.append(_xpeak)
        ypeak_in_nonsubdivided.append(_ypeak)
        ra_in_nonsubdivided.append((celestial_coords.ra * u.deg).value)
        dec_in_nonsubdivided.append((celestial_coords.dec * u.deg).value)

    # Note Caveat: XWIN_IMAGE, YWIN_IMAGE, X_IMAGE, and Y_IMAGE are all overwritten using X_IMAGE_DBL and Y_IMAGE_DBL.
    objects['XWIN_IMAGE'] = x_in_nonsubdivided
    objects['YWIN_IMAGE'] = y_in_nonsubdivided
    objects['X_IMAGE'] = x_in_nonsubdivided
    objects['Y_IMAGE'] = y_in_nonsubdivided
    objects['X_IMAGE_DBL'] = x_in_nonsubdivided
    objects['Y_IMAGE_DBL'] = y_in_nonsubdivided
    objects['XWIN_WORLD'] = ra_in_nonsubdivided
    objects['YWIN_WORLD'] = dec_in_nonsubdivided
    objects['X_WORLD'] = ra_in_nonsubdivided
    objects['Y_WORLD'] = dec_in_nonsubdivided
    objects['XPEAK_IMAGE'] = xpeak_in_nonsubdivided
    objects['YPEAK_IMAGE'] = ypeak_in_nonsubdivided

    # Indicator to inform which subdivision were these objects from.
    objects['SUBDIV_NUMBER'] = [i] * len(objects)
    objects['SUBDIV_NUMBER'] = objects['SUBDIV_NUMBER'].astype(int)

    if len(objects) == 1 and objects['NUMBER'].iloc[0] == -99:
        # No source is detected, so `objects` is dummy. So don't add it in detected catalog.
        assert orig_fluxes_subdiv == np.sum(subdiv.data - orig_bkg)
    else:
        orig_objects.append(np.expand_dims(objects, 1))

orig_objects = np.squeeze(np.vstack(orig_objects))

if use_sextractor and use_subdiv:
    CAT_COLUMNS.append('SUBDIV_NUMBER')  # Because we added `SUBDIV_NUMBER` when using subdivisions.
    # pd.DataFrame(data=orig_objects, columns=columns).to_csv(f'{os.path.join(dirname, f"orig_{basename}")}_scat_sextractor.csv')
    # pd.DataFrame(data=deconv_objects, columns=columns).to_csv(f'{os.path.join(dirname, f"deconv_{basename}")}_scat_sextractor.csv')

    _o = pd.DataFrame(data=orig_objects, columns=CAT_COLUMNS)
    _o.columns = CAT_COLUMNS
    _o.to_csv(f'{os.path.join(dirname, f"ref_{basename}")}_scat_sextractor.csv')
    
    # Once the final catalog is created, remove all subdivision files to clean up space.
    for img in glob.glob(os.path.join(dirname, 'subdiv*.fits')):
        os.remove(img)
    for img in glob.glob(os.path.join(dirname, 'subdiv*.csv')):
        os.remove(img)
    
    # If there is some overlap or if the entire image size is not an integral multiple of the subdivision size, same sources can be detected across two adjacent subdivisions. Here, we remove duplicate rows.
    # Note that, in the current implementation, each subdivision will be a square irrespective of whether the image size is an integral multiple of the subdivision size or not.
    # This means that in such a case, the subdivisions will have an overlap even if subdiv_overlap=0.
    # Practically, it's very common that the image size is not an integral multiple of the subdivision size.
    _orig_source_cat = pd.read_csv(f'{os.path.join(dirname, f"ref_{basename}")}_scat_sextractor.csv')
    _num_orig_sources = _orig_source_cat.shape[0]

    # Set object IDs manually since the IDs were based on subdivisions only. To make them unique, we need to do the below preprocessing.
    def set_id(old_id, subdiv_number):
        return f'{int(subdiv_number)}_{int(old_id)}'
    _orig_source_cat['NUMBER'] = [set_id(_orig_source_cat.loc[idx, 'NUMBER'], _orig_source_cat.loc[idx, 'SUBDIV_NUMBER']) for idx in range(len(_orig_source_cat))]
    _orig_source_cat['ID_PARENT'] = [set_id(_orig_source_cat.loc[idx, 'ID_PARENT'], _orig_source_cat.loc[idx, 'SUBDIV_NUMBER']) for idx in range(len(_orig_source_cat))]

    # Remove redundant rows, i.e. rows with the same (x, y) location of the source.
    # Note that we are using the X and Y coordinates instead of ID_PARENT/NUMBER because the aim of removing these duplicates is that since we have subdivision overlap, coinciding sources
    # from adjacent subdivisions will be detected twice. We can only use coordinates to remove them since because they are in different subdivisions, ID_PARENT/NUMBER will never be the same, so it cannot be used for removal.
    # 1. FIRST FILTERING (using sub-pixel-precision coordinates).
    _orig_source_cat.drop_duplicates(subset=['X_IMAGE_DBL', 'Y_IMAGE_DBL'], inplace=True, keep='first')  # Only the first occurrence is retained.
    # 2. SECOND FILTERING (using integer coordinates).
    # This second filtering is used because two sources from different subdivisions may correspond to the same actual source (even if subdiv_overlap=0)
    # So, this condition will remove such duplicates. The first condition above removes exact duplicates, but this condition tries to remove near-duplicates.
    # Note that we don't check the flux/mag of the sources before removing duplicates: ideally, we only want to remove those whose XPEAK and YPEAK match but also their magnitudes
    # to increase the chances of knowing it's the same source. Doing that is more trustworthy, but what we do below may also be sufficient and okay.
    _orig_source_cat.drop_duplicates(subset=['XPEAK_IMAGE', 'YPEAK_IMAGE'], inplace=True, keep='first')  # Only the first occurrence is retained.
    _num_orig_sources_after = _orig_source_cat.shape[0]

    # 3. THIRD FILTERING. (This filtering is the most strongest and can be replaced with the previous two conditions,
    # but just using this as a third condition for transparency).
    _orig_source_cat = remove_very_close_coords(_orig_source_cat, threshold=1.0)

    print(f'[Original]: {_num_orig_sources_after} sources out of {_num_orig_sources} remaining after removing duplicates')

    # Note that while the above handles cases where the coordinates in two rows exactly match, sometimes the coordinates are not exact but instead equal up to an error tolerance.
    # For matching for each row in Table 2, for example, ideally we want these two subsets of the same shape: df['X_IMAGE_DBL_2'].unique().shape, df['X_IMAGE_DBL_2'].shape. Same for y-coordinates.
    # But if these two are not of the same shape, then in this code this means that the coordinates were almost equal (within some tolerance) but not exactly the same, and that other source properties match.
    # However, such cases are delicate since it might happen these sources are very nearby (blended).
    # So NOTE: in this code, we refrain from manipulating such cases which means (for crossmatching for each row in Table 2), df['X_IMAGE_DBL_2'].unique().shape, df['X_IMAGE_DBL_2'].shape may not be the same.
    # This means the same original source will kind of repeated. But the no. of such cases is not too high since we have atleast removed the exact duplicate rows above.
    # Now write to file.

    # Remove rows in the original source catalog that have area < 5.
    _orig_source_cat = _orig_source_cat.drop(_orig_source_cat[_orig_source_cat.ISOAREAF_IMAGE < 5].index)
    print(f'{_orig_source_cat.shape[0]} rows remaining out of {_num_orig_sources_after} rows in the original catalog after filtering out rows with ISOAREAF < 5')

    _orig_source_cat.to_csv(f'{os.path.join(dirname, f"ref_{basename}")}_scat_sextractor.csv')

if use_subdiv:
    # Note: If we use the subdivision approach, then the below type of background files are not needed anymore.
    for img in glob.glob('*.fits_scat_sextractor_bkg.fits'):
        os.remove(img)

# Remove temporary orig and deconvolved images.
for img in glob.glob(f'{dirname}/temp_deconvolved_*.fits'):
    os.remove(img)
for img in glob.glob(f'{dirname}/temp_orig_*.fits'):
    os.remove(img)
if use_sextractor:
    for img in glob.glob(f'{dirname}/subdiv_*_temp.fits'):
        os.remove(img)


import subprocess
os.chdir('/kaggle/working/sgp_reconstruction_results')
subprocess.run(
    [
        './topcat', '-stilts', 'tmatch2', f'in1=deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv', 'ifmt1=csv',
        f'in2=ref_{basename}_scat_sextractor.csv', 'ifmt2=csv',
        f'out={basename}_crossmatched_best_match_for_each_table1_row_1_and_2_stilts_ref.csv',
        'matcher=sky', 'values1=X_WORLD Y_WORLD', 'values2=X_WORLD Y_WORLD',
        'join=1and2', 'params=1.399', 'find=best1', 'progress=time'  # 1.399 = 1.383*1.012
    ]
)

subprocess.run(
    [
        './topcat', '-stilts', 'tmatch2', f'in1=deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv', 'ifmt1=csv',
        f'in2=ref_{basename}_scat_sextractor.csv', 'ifmt2=csv',
        f'out={basename}_crossmatched_best_match_for_each_table2_row_1_and_2_stilts_ref.csv',
        'matcher=sky', 'values1=X_WORLD Y_WORLD', 'values2=X_WORLD Y_WORLD',
        'join=1and2', 'params=1.399', 'find=best2', 'progress=time'
    ]
)

subprocess.run(
    [
        './topcat', '-stilts', 'tmatch2', f'in1=deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv', 'ifmt1=csv',
        f'in2=ref_{basename}_scat_sextractor.csv', 'ifmt2=csv',
        f'out={basename}_crossmatched_best_match_symmetric_1_and_2_stilts_ref.csv',
        'matcher=sky', 'values1=X_WORLD Y_WORLD', 'values2=X_WORLD Y_WORLD',
        'join=1and2', 'params=1.399', 'find=best', 'progress=time'
    ]
)

subprocess.run(
    [
        './topcat', '-stilts', 'tmatch2', f'in1=deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv', 'ifmt1=csv',
        f'in2=ref_{basename}_scat_sextractor.csv', 'ifmt2=csv',
        f'out={basename}_crossmatched_all_matches_1_and_2_stilts_ref.csv',
        'matcher=sky', 'values1=X_WORLD Y_WORLD', 'values2=X_WORLD Y_WORLD',
        'join=1and2', 'params=1.399', 'find=all', 'progress=time'
    ]
)

subprocess.run(
    [
        './topcat', '-stilts', 'tmatch2', f'in1=deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv', 'ifmt1=csv',
        f'in2=ref_{basename}_scat_sextractor.csv', 'ifmt2=csv',
        f'out={basename}_crossmatched_best_match_for_each_table1_row_1_not_2_stilts_ref.csv',
        'matcher=sky', 'values1=X_WORLD Y_WORLD', 'values2=X_WORLD Y_WORLD',
        'join=1not2', 'params=1.399', 'find=best1', 'progress=time'
    ]
)

subprocess.run(
    [
        './topcat', '-stilts', 'tmatch2', f'in1=deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv', 'ifmt1=csv',
        f'in2=ref_{basename}_scat_sextractor.csv', 'ifmt2=csv',
        f'out={basename}_crossmatched_best_match_for_each_table2_row_1_not_2_stilts_ref.csv',
        'matcher=sky', 'values1=X_WORLD Y_WORLD', 'values2=X_WORLD Y_WORLD',
        'join=1not2', 'params=1.399', 'find=best2', 'progress=time'
    ]
)

subprocess.run(
    [
        './topcat', '-stilts', 'tmatch2', f'in1=deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv', 'ifmt1=csv',
        f'in2=ref_{basename}_scat_sextractor.csv', 'ifmt2=csv',
        f'out={basename}_crossmatched_best_match_for_each_table2_row_2_not_1_stilts_ref.csv',
        'matcher=sky', 'values1=X_WORLD Y_WORLD', 'values2=X_WORLD Y_WORLD',
        'join=2not1', 'params=1.399', 'find=best2', 'progress=time'
    ]
)

In [52]:
!zip -r deepref_sci_compare_cats.zip *.csv

  adding: deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv (deflated 70%)
  adding: orig_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv (deflated 65%)
  adding: ref_ztf_000626_zr_c12_q2_deeprefimg_align1.fits_scat_sextractor.csv (deflated 66%)
  adding: ztf_000626_zr_c12_q2_deeprefimg_align1.fits_crossmatched_all_matches_1_and_2_stilts_ref.csv (deflated 66%)
  adding: ztf_000626_zr_c12_q2_deeprefimg_align1.fits_crossmatched_best_match_for_each_table1_row_1_and_2_stilts_ref.csv (deflated 66%)
  adding: ztf_000626_zr_c12_q2_deeprefimg_align1.fits_crossmatched_best_match_for_each_table1_row_1_not_2_stilts_ref.csv (deflated 72%)
  adding: ztf_000626_zr_c12_q2_deeprefimg_align1.fits_crossmatched_best_match_for_each_table2_row_1_and_2_stilts_ref.csv (deflated 66%)
  adding: ztf_000626_zr_c12_q2_deeprefimg_align1.fits_crossmatched_best_match_for_each_table2_row_1_not_2_stilts_ref.csv (deflated 72%)
  adding: ztf_000626_zr_c12_q2_deeprefimg_al

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name):
    zip_name = f"/kaggle/working/sgp_reconstruction_results/{download_file_name}.zip"
    display(FileLink(f'{download_file_name}.zip'))

download_file('ref_sci_compare_cats')

In [41]:
!ls /kaggle/working/sgp_reconstruction_results/*.zip

/kaggle/working/sgp_reconstruction_results/ref_sci_compare_cats.zip


## richardson-lucy

In [46]:
from subprocess import Popen, PIPE, CalledProcessError, STDOUT
def execute(cmd):
    """This function is taken from
    https://stackoverflow.com/a/28319191
    """
#     logfile = open('sgp_run_all.log', 'a')
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True, stderr=STDOUT) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
#         print(p.output)
        raise CalledProcessError(p.returncode, p.args)

for prefix in prefixes:
    if 'ztf_20190629424757_000594_zr_c04_o_q1_' in prefix:  # Means it's a globular cluster. We are using the same command as the non-globular cluster case (previously I used subdiv_size=128).
        command = [
            'python3', 'run.py', '--data_path_sciimg', f'/kaggle/working/{prefix.split("/")[-1]}sciimg.fits',
            '--data_path_psf', f'/kaggle/working/{prefix.split("/")[-1]}sciimgdaopsfcent.fits',
            '--init_recon', '2', '--stop_criterion', '3', '--use_subdiv', '--subdiv_size', '512', '--subdiv_overlap', '10',
            '--use_sextractor', '--sextractor_config_file_name', 'sextractor_config.fits_scat_sextractor.sex',
            '--use_subdiv', '--interpolate_bad_pixels', '--reconstruct_full_image_from_subdivisions',
            '--tol_convergence', '1e-4', '--use_rl',
            # '--pixel_mask', '/kaggle/working/ztf_20180831500891_000251_zr_c16_o_q4_mskimg.fits',  # todo: how to use this?
            '--perform_catalog_crossmatching'
        ]
    else:
        command = [
            'python3', 'run.py', '--data_path_sciimg', f'/kaggle/working/{prefix.split("/")[-1]}sciimg.fits',
            '--data_path_psf', f'/kaggle/working/{prefix.split("/")[-1]}sciimgdaopsfcent.fits',
            '--init_recon', '2', '--stop_criterion', '3', '--use_subdiv', '--subdiv_size', '512', '--subdiv_overlap', '10',
            '--use_sextractor', '--sextractor_config_file_name', 'sextractor_config.fits_scat_sextractor.sex',
            '--use_subdiv', '--interpolate_bad_pixels', '--reconstruct_full_image_from_subdivisions',
            '--tol_convergence', '1e-4', '--use_rl',
            # '--pixel_mask', '/kaggle/working/ztf_20180831500891_000251_zr_c16_o_q4_mskimg.fits',  # todo: how to use this?
            '--perform_catalog_crossmatching'
        ]

    execute(command)



------------ Options ------------
data_path_sciimg:		/kaggle/working/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits
data_path_psf:		/kaggle/working/ztf_20221120493183_000626_zr_c12_o_q2_sciimgdaopsfcent.fits
scale_psf:		False
psf_scale_fwhm:		1.2
init_recon:		2
stop_criterion:		3
flip_image:		False
box_height:		64
box_width:		64
use_subdiv:		True
subdiv_size:		512
subdiv_overlap:		10
sextractor_config_file_name:		sextractor_config.fits_scat_sextractor.sex
use_sextractor:		True
use_rl:		True
use_beta_div:		False
initial_beta:		1.005
initial_lr:		0.001
tol_convergence:		0.0001
gain:		None
saturate:		None
reconstruct_full_image_from_subdivisions:		True
reconstruct_subdivisions_fast:		False
interpolate_bad_pixels:		True
pixel_mask:		
perform_catalog_crossmatching:		True
------------ End ----------------


['source-extractor', 'subdiv_0_temp.fits', '-c', 'sextractor_config.fits_scat_sextractor.sex', '-CATALOG_NAME', 'subdiv_orig_0_251.fits_scat_sextractor.csv', '-CHECKIMAGE_NAME', 'sub

In [37]:
# !rm -f sgp_reconstruction_results/*.fits sgp_reconstruction_results/*.csv

In [48]:
!zip -r RL_626r_results_spatialReg.zip sgp_reconstruction_results/*.csv sgp_reconstruction_results/deconvolved_subdiv*.fits sgp_reconstruction_results/orig_subdiv*.fits

updating: sgp_reconstruction_results/deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv (deflated 66%)
updating: sgp_reconstruction_results/orig_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv (deflated 65%)
updating: sgp_reconstruction_results/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_crossmatched_all_matches_1_and_2_stilts.csv (deflated 65%)
updating: sgp_reconstruction_results/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_crossmatched_best_match_for_each_table1_row_1_and_2_stilts.csv (deflated 65%)
updating: sgp_reconstruction_results/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_crossmatched_best_match_for_each_table1_row_1_not_2_stilts.csv (deflated 66%)
updating: sgp_reconstruction_results/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_crossmatched_best_match_for_each_table2_row_1_and_2_stilts.csv (deflated 65%)
updating: sgp_reconstruction_results/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_crossmatched_bes

In [17]:
!ls -lh *.zip

-rw-r--r-- 1 root root 297M Sep 25 12:14 RL_626r_829r_results.zip


In [18]:
!ls -lh sgp_reconstruction_results/*

-rw-r--r-- 1 root root  37M Sep 25 11:34 sgp_reconstruction_results/deconv_bkg_ztf_20220706370127_000829_zr_c10_o_q4_sciimg.fits
-rw-r--r-- 1 root root  37M Sep 25 12:11 sgp_reconstruction_results/deconv_bkg_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits
-rw-r--r-- 1 root root  37M Sep 25 11:34 sgp_reconstruction_results/deconv_bkgrms_ztf_20220706370127_000829_zr_c10_o_q4_sciimg.fits
-rw-r--r-- 1 root root  37M Sep 25 12:11 sgp_reconstruction_results/deconv_bkgrms_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits
-rw-r--r-- 1 root root 151M Sep 25 11:34 sgp_reconstruction_results/deconv_ztf_20220706370127_000829_zr_c10_o_q4_sciimg.fits_scat_sextractor.csv
-rw-r--r-- 1 root root  69M Sep 25 12:11 sgp_reconstruction_results/deconv_ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits_scat_sextractor.csv
-rw-r--r-- 1 root root  37M Sep 25 11:34 sgp_reconstruction_results/deconvolved_subdiv_ztf_20220706370127_000829_zr_c10_o_q4_sciimg.fits
-rw-r--r-- 1 root root  37M Sep 25 12:11 sgp_recons